In [1]:
import numpy as np
import pandas as pd
from pymongo import MongoClient
import json
import urllib.parse
import bson

In [2]:
nobel = pd.read_html('https://en.wikipedia.org/wiki/List_of_Nobel_laureates',header=0)[0]

In [3]:
nobel.head()

,Year,Physics,Chemistry,Physiologyor Medicine,Literature,Peace,Economics
0,1901,Wilhelm Röntgen,Jacobus Henricus van 't Hoff,Emil Adolf von Behring,Sully Prudhomme,Henry Dunant;Frédéric Passy,—
1,1902,Hendrik Lorentz;Pieter Zeeman,Hermann Emil Fischer,Ronald Ross,Theodor Mommsen,Élie Ducommun;Charles Albert Gobat,—
2,1903,Henri Becquerel;Pierre Curie;Marie Curie,Svante Arrhenius,Niels Ryberg Finsen,Bjørnstjerne Bjørnson,Randal Cremer,—
3,1904,Lord Rayleigh,William Ramsay,Ivan Pavlov,Frédéric Mistral;José Echegaray,Institut de Droit International,—
4,1905,Philipp Lenard,Adolf von Baeyer,Robert Koch,Henryk Sienkiewicz,Bertha von Suttner,—


In [4]:
nobel['Year'] = pd.to_numeric(nobel['Year'],errors='coerce')

In [5]:
nobel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 7 columns):
Year                     116 non-null float64
Physics                  118 non-null object
Chemistry                118 non-null object
Physiologyor Medicine    118 non-null object
Literature               118 non-null object
Peace                    118 non-null object
Economics                118 non-null object
dtypes: float64(1), object(6)
memory usage: 6.5+ KB


In [6]:
nobel_json1 = nobel.T.to_json()

In [7]:
records = json.loads(nobel_json1).values()

In [9]:
type(records)

dict_values

In [11]:
help(MongoClient())

Help on MongoClient in module pymongo.mongo_client object:

class MongoClient(pymongo.common.BaseObject)
 |  A base class that provides attributes and methods common
 |  to multiple pymongo classes.
 |  
 |  SHOULD NOT BE USED BY DEVELOPERS EXTERNAL TO MONGODB.
 |  
 |  Method resolution order:
 |      MongoClient
 |      pymongo.common.BaseObject
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __exit__(self, exc_type, exc_val, exc_tb)
 |  
 |  __getattr__(self, name)
 |      Get a database by name.
 |      
 |      Raises :class:`~pymongo.errors.InvalidName` if an invalid
 |      database name is used.
 |      
 |      :Parameters:
 |        - `name`: the name of the database to get
 |  
 |  __getitem__(self, name)
 |      Get a database by name.
 |      
 |      Raises :class:`~pymongo.errors.InvalidName` if an invalid
 |      database name is used.
 |      
 |      :Parameters:
 |      

In [20]:
uri = "mongodb://%s:%s@%s/%s" % (urllib.parse.quote_plus('mariano'), urllib.parse.quote_plus('federico'), 'localhost','nobel_prizes')

conn = MongoClient(uri)

In [21]:
db = conn.nobel_prizes

In [25]:
collection = db['winners']

In [26]:
collection.insert_many(records)

In [27]:
resultado = collection.find()

In [28]:
resultado

In [11]:
# check information provided by objectID

Object = list(resultado)[0]

In [12]:
oid = bson.ObjectId(Object['_id'])
oid.generation_time

datetime.datetime(2018, 6, 15, 12, 2, 36, tzinfo=<bson.tz_util.FixedOffset object at 0x000001E806878198>)

In [13]:
Object

{'_id': ObjectId('5b23aadc949ebb2cf000e4bf'),
 'Year': '1901',
 'Physics': 'Röntgen, WilhelmWilhelm Röntgen',
 'Chemistry': "Hoff, Jacobus Henricus van 'tJacobus Henricus van 't Hoff",
 'Physiology or Medicine': 'von Behring, Emil AdolfEmil Adolf von Behring',
 'Literature': 'Prudhomme, SullySully Prudhomme',
 'Peace': 'Dunant, HenryHenry Dunant; Passy, FrédéricFrédéric Passy',
 'Economics': '—'}

### Searching for Data on MongoDB

In [14]:
res = collection.find({'Year':{'$gt':1940}})
list(res)

[]